In [20]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

import pandas as pd
import scipy
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle


In [7]:
import os
import pandas as pd
import soundfile as sf
import numpy as np
import math
import scipy.io.wavfile, scipy.signal
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier



cols=['video_id','start_time','mid_ts','label','audio','vggish']

d=np.load('/Users/vedant/Desktop/Programming/ScreamDetection/resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv('/Users/vedant/Desktop/Programming/ScreamDetection/resources/dataset/lookup_new.csv')





In [8]:
feature_df=df[cols]
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(2)
    if row['label'] == 'lowfry':
        mapping.append(3)
    if row['label'] == 'midfry':
        mapping.append(4)
    if row['label'] == 'no_vocals':
        mapping.append(5)

feature_df.insert(4,'label_mapped',mapping)

In [9]:
feature_df

,video_id,start_time,mid_ts,label,label_mapped,audio,vggish
0,4600fGWcn9o,0.0,0.5,no_vocals,5,"[0.0, 0.0, 0.0, -3.0517578125e-05, -1.52587890...","[166.0, 8.0, 149.0, 128.0, 199.0, 57.0, 96.0, ..."
1,4600fGWcn9o,0.5,1.0,no_vocals,5,"[0.0004730224609375, 0.0001983642578125, -6.10...","[175.0, 10.0, 147.0, 103.0, 210.0, 74.0, 81.0,..."
2,4600fGWcn9o,1.0,1.5,no_vocals,5,"[-4.57763671875e-05, -3.0517578125e-05, 1.5258...","[173.0, 10.0, 148.0, 131.0, 191.0, 76.0, 71.0,..."
3,4600fGWcn9o,1.5,2.0,no_vocals,5,"[3.0517578125e-05, 0.0, -3.0517578125e-05, 0.0...","[164.0, 7.0, 154.0, 128.0, 190.0, 80.0, 29.0, ..."
4,4600fGWcn9o,2.0,2.5,no_vocals,5,"[-0.0008087158203125, -0.000885009765625, -0.0...","[169.0, 11.0, 146.0, 115.0, 191.0, 75.0, 108.0..."
...,...,...,...,...,...,...,...
33815,0m5fIHHfJTw,217.5,218.0,no_vocals,5,"[1.52587890625e-05, 6.103515625e-05, 0.0001220...","[174.0, 13.0, 142.0, 130.0, 194.0, 76.0, 84.0,..."
33816,0m5fIHHfJTw,218.0,218.5,no_vocals,5,"[6.103515625e-05, 9.1552734375e-05, 0.00012207...","[174.0, 10.0, 148.0, 124.0, 190.0, 77.0, 86.0,..."
33817,0m5fIHHfJTw,218.5,219.0,no_vocals,5,"[0.0, -1.52587890625e-05, -1.52587890625e-05, ...","[171.0, 9.0, 140.0, 110.0, 201.0, 56.0, 68.0, ..."
33818,0m5fIHHfJTw,219.0,219.5,no_vocals,5,"[-1.52587890625e-05, -1.52587890625e-05, -1.52...","[172.0, 10.0, 141.0, 109.0, 203.0, 56.0, 71.0,..."


In [10]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='not minority',random_state=0)
from collections import Counter
X = feature_df[cols].to_numpy()
y=feature_df[['label_mapped']].to_numpy()

X_under, y_under = undersample.fit_resample(X, y)

undersampled_data = pd.DataFrame(X_under,columns=cols)
undersampled_data['label_mapped'] = y_under
#print(undersampled_data)


from sklearn.model_selection import GroupShuffleSplit
train_inds, test_inds = next(GroupShuffleSplit(test_size=.2, n_splits=2, random_state = 0).split(lut, groups=lut['band_name']))

train = lut.iloc[train_inds]
test = lut.iloc[test_inds]

train_ids = train['video_id'].to_numpy()
test_ids = test['video_id'].to_numpy()

#df_final = df
df_final = undersampled_data
train = df_final[df_final.video_id.isin(train_ids)]
test = df_final[df_final.video_id.isin(test_ids)]

In [12]:
x_train = train['audio'].to_numpy()
y_train_hot = to_categorical(train['label_mapped'].to_numpy())

x_test = test['audio'].to_numpy()
y_test_hot = to_categorical(test['label_mapped'].to_numpy())

In [13]:
import librosa

In [27]:
X_train = []
X_test = []
for y in x_train:
    X_train.append(librosa.feature.melspectrogram(y=y, sr=44100))
for y in x_test:
    X_test.append(librosa.feature.melspectrogram(y=y, sr=44100))

In [25]:
np.abs(librosa.stft(y,44100))

array([[3.67109888e-01, 3.56413117e-01, 3.41168868e-01, 1.64408049e+00,
        3.62421605e+00],
       [1.89459790e-01, 1.78043458e-01, 1.68788214e-01, 1.98767356e+00,
        3.79144133e+00],
       [7.20001834e-03, 4.69622717e-03, 1.60461646e-03, 1.97867469e+00,
        3.95877778e+00],
       ...,
       [1.82328636e-02, 8.86772891e-03, 1.40041148e-04, 4.46740574e-03,
        8.68626982e-03],
       [1.90051444e-02, 9.63314786e-03, 4.09516404e-04, 4.54941867e-03,
        8.84801564e-03],
       [1.95289571e-02, 9.90402639e-03, 5.63061428e-05, 4.14994282e-03,
        8.69031545e-03]])

In [34]:
X_train_stft = []
X_test_stft = []
for y in x_train:
    X_train_stft.append(np.abs(librosa.stft(y)))
for y in x_test:
    X_test_stft.append(np.abs(librosa.stft(y)))

In [48]:
x_train.shape

(2280,)

In [36]:
print(x_test.shape)
print(np.array(X_test_stft).shape)

(1050,)
(1050, 1025, 87)


In [46]:
np.array(X_test)[0].shape

(128, 87)

In [54]:
X_train=np.array(X_train)
X_test=np.array(X_test)

X_train = X_train.reshape(X_train.shape[0], 128, 87, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 87, 1)

X_train,y_train=shuffle(X_train,y_train_hot)
X_test,y_test=shuffle(X_test,y_test_hot)

In [37]:
X_train_stft=np.array(X_train_stft)
X_test_stft=np.array(X_test_stft)

X_train_stft = X_train_stft.reshape(X_train_stft.shape[0], 1025, 87, 1)
X_test_stft = X_test_stft.reshape(X_test_stft.shape[0], 1025, 87, 1)

X_train_stft,y_train_stft=shuffle(X_train_stft,y_train_hot)
X_test_stft,y_test_stft=shuffle(X_test_stft,y_test_hot)

In [50]:
# model
model = Sequential()

input_shape = (1025,87,1)#(128, 87, 1)
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(512,43,8)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 1024, 86, 16)      80        
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 512, 43, 16)       0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 352256)            0         
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 2113542   
Total params: 2,113,622
Trainable params: 2,113,622
Non-trainable params: 0
_________________________________________________________________


# Mel spectrogram + 1 layer

In [60]:
# model
model = Sequential()

input_shape = (128, 87, 1)
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(512,43,8)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
epochs=10
batch_size=128
# fit the model
model.fit(np.array(X_train), y_train_hot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_hot))

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 127, 86, 16)       80        
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 63, 43, 16)        0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 43344)             0         
_________________________________________________________________
dense_28 (Dense)             (None, 6)                 260070    
Total params: 260,150
Trainable params: 260,150
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
18/18 [==============================] - 4s 209ms/step - loss: 7.2577 - accuracy: 0.1741 - val_loss: 6.2692 - val_accuracy: 0.1695
Epoch 2/10
18/18 [==============================] - 3s 179ms/step - loss: 7.1166 - accuracy: 0.1746 - v

# Mel Spectrogram + 2 layers

In [56]:
# model
model = Sequential()

input_shape = (128, 87, 1)
model.add(Conv2D(8, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(512,43,8)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
epochs=10
batch_size=128
# fit the model
model.fit(np.array(X_train), y_train_hot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_hot))

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 127, 86, 8)        40        
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 63, 43, 8)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 62, 42, 16)        528       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 31, 21, 16)        0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 10416)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               1333376   
_________________________________________________________________
dense_23 (Dense)             (None, 6)               

# STFT + 1 conv layer

In [58]:
# model
model = Sequential()

input_shape = (1025,87,1)
model.add(Conv2D(8, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(512,43,8)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
epochs=10
batch_size=128
# fit the model
model.fit(np.array(X_train_stft), y_train_hot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_stft, y_test_hot))

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 1024, 86, 8)       40        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 512, 43, 8)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 176128)            0         
_________________________________________________________________
dense_25 (Dense)             (None, 6)                 1056774   
Total params: 1,056,814
Trainable params: 1,056,814
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
18/18 [==============================] - 16s 799ms/step - loss: 2.3863 - accuracy: 0.1781 - val_loss: 2.3159 - val_accuracy: 0.1724
Epoch 2/10
18/18 [==============================] - 11s 607ms/step - loss: 2.2547 - accuracy: 0.17

# STFT + 2 conv layers

In [59]:
# model
model = Sequential()

input_shape = (1025,87,1)
model.add(Conv2D(8, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', input_shape=(512,43,8)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
epochs=10
batch_size=128
# fit the model
model.fit(np.array(X_train_stft), y_train_hot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_stft, y_test_hot))

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 1024, 86, 8)       40        
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 512, 43, 8)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 511, 42, 16)       528       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 255, 21, 16)       0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 85680)             0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               10967168  
_________________________________________________________________
dense_27 (Dense)             (None, 6)               